In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [3]:
import pprint

import torch
import pickle
from PIL import Image
import numpy as np
from pytorch3d.renderer import (
    look_at_view_transform
)
import matplotlib.pyplot as plt
import glob

from pose_est import brute_force_pose_est
import postprocess_dataset
from utils import utils

In [4]:
# displays meshes at the predicted pose
def show_meshes(input_dir_img, input_dir_mesh, meshes_group_name=""):
    pred_poses_path = os.path.join(input_dir_mesh, "pred_poses.p")
    cached_pred_poses = pickle.load(open(pred_poses_path, "rb"))

    for instance_name in cached_pred_poses:
        input_image = Image.open(os.path.join(input_dir_img, instance_name+".png"))
        with torch.no_grad():
            if meshes_group_name == "":
                mesh_filename_suffix = '.obj'
            else:
                mesh_filename_suffix = "_{}.obj".format(meshes_group_name)
            mesh = utils.load_untextured_mesh(os.path.join(input_dir_mesh, instance_name+mesh_filename_suffix), device)

        # rendering mesh at predicted pose
        pred_dist = cached_pred_poses[instance_name]['dist']
        pred_elev = cached_pred_poses[instance_name]['elev']
        pred_azim = cached_pred_poses[instance_name]['azim']
        R, T = look_at_view_transform(pred_dist, pred_elev, pred_azim) 
        mesh_rendered_at_pred_pose = utils.render_mesh(mesh, R, T, device)

        # visualizing
        plt.imshow(input_image)
        plt.show()
        plt.imshow(mesh_rendered_at_pred_pose[0, ..., :3].cpu().numpy())
        plt.show()

In [5]:
input_dir_img = "data/test_dataset/"
input_dir_mesh = "data/test_dataset/"
cfg_path = "configs/default.yaml"
gpu_num = 0
device = torch.device("cuda:"+str(gpu_num))
experiment_name = "num_azim_0"

In [ ]:
dataset_loss_info = postprocess_dataset.postprocess_data(input_dir_img, input_dir_mesh, cfg_path, gpu_num, meshes_group_name=experiment_name, recompute_poses=False)

/home/svcl-oowl/brandon/research/sil_consistent_at_inference/utils/utils.py:51: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg_special = yaml.load(f)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
# show postprocessed meshes at predicted angle
show_meshes(input_dir_img, input_dir_mesh, experiment_name)

In [ ]:
# show training loss info
for instance_name in dataset_loss_info:
    loss_info = dataset_loss_info[instance_name]
    loss_info.plot.line(x='iter', y='total_loss', title="{}".format(instance_name))
    #loss_info.plot.line(x='iter', y='sil_loss')
    #loss_info.plot.line(x='iter', y='img_sym_loss')
    #loss_info.plot.line(x='iter', y='vertex_sym_loss')
    #loss_info.plot.line(x='iter', y='l2_loss')
    #loss_info.plot.line(x='iter', y='lap_smoothness_loss')
    #loss_info.plot.line(x='iter', y='normal_consistency_loss')

In [ ]:
# show original meshes at predicted angle
show_meshes(input_dir_img, input_dir_mesh)

In [ ]:
# delete all postprocessed meshes, reverting back to only original objs and imgs
all_mesh_paths = glob.glob(os.path.join(input_dir_mesh, "*.obj"))
for mesh_path in all_mesh_paths:
    if '_' in mesh_path.split('/')[-1]:
        os.remove(mesh_path)